In [0]:
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

1.1 Implement gradient-based factorisation (1 mark)

In [0]:
from typing import Tuple

# A = torch.Tensor([[5,3,0,1], [4,0,0,1], [1,1,0,5], [1,0,0,4],[0,1,5,4]])
# m, n = A.shape
# r = 3
# N = 1000
# lr = 0.01

def sgd_factorise(A, r, N, lr):
  # initialised
  U = torch.rand(m, r)
  V = torch.rand(n, r)
   
  for epoch in range(N):
    # iterative update
    for r in range(m):
      # row
      for c in range(n):
        # column
        e = A[r, c] - U[r] @ V[c].t()
        U[r] = U[r] + lr*e*V[c]
        V[c] = V[c] + lr*e*U[r]
  return U, V

# U, V = sgd_factorise(A,r,N,lr)
# print(U)
# print(V)

1.2 Factorise and compute reconstruction error (1 mark)

In [27]:
A = torch.Tensor([[0.3374,0.6005,0.1735], 
                  [3.3359,0.0492,1.8374], 
                  [2.9407,0.5301,2.2620]])
m, n = A.shape
r = 2
N = 1000
lr = 0.01

U, V = sgd_factorise(A,r,N,lr)
print("U:")
print(U)
print("V:")
print(V)

U:
tensor([[-0.0704,  0.3450],
        [ 0.9041,  1.6369],
        [ 1.4271,  1.1171]])
V:
tensor([[0.9390, 1.4798],
        [0.0732, 0.1708],
        [1.1493, 0.5188]])


In [28]:
# compute reconstruction loss
# e1 = (A - U@V.t())**2
# error1 = torch.sum(e1)
# print(error1)
loss_fn = torch.nn.MSELoss(reduction = 'sum')
loss1 = loss_fn(A, U@V.t())
print(loss1)

tensor(0.4705)


2.1 Compare to the truncated-SVD (1 mark)

In [29]:
u, s, v =torch.svd(A)
# set the last singular value to zero 
s[2]=0
# compute the reconstruction
A1 = torch.mm(torch.mm(u, torch.diag(s)), v.t())
print(A1)

# compute error
# e2 = (A-A1)**2
# error2 = torch.sum(e2)
# print(error2)
loss2 = loss_fn(A, A1)
print(loss2)

tensor([[ 0.2245,  0.5212,  0.3592],
        [ 3.2530, -0.0090,  1.9737],
        [ 3.0378,  0.5983,  2.1023]])
tensor(0.1219)


3.1 Implement masked factorisation (1 mark)

In [0]:
def sgd_factorise_masked(A, M, r, N, lr):
  # initialised
  U = torch.rand(m, r)
  V = torch.rand(n, r)

  for epoch in range(N):
    # iterative update
    for r in range(m):
      # row
      for c in range(n):        
        # column
        if M[r,c]=='1':
          # if value is valid
          e = A[r, c] - U[r] @ V[c].t()
          U[r] = U[r] + lr*e*V[c]
          V[c] = V[c] + lr*e*U[r]
        # elif M[r,c]=='0':
        #   pass
  return U, V

3.2 Reconstruct a matrix (1 mark)

In [35]:
A = torch.Tensor([[0.3374,0.6005,0.1735], 
                  [0,0.0492,1.8374], 
                  [2.9407,0,2.2620]])
M = torch.Tensor([[1,1,1],
                  [0,1,1],
                  [1,0,1]])
m, n = A.shape
r = 2
N = 1000
lr = 0.01

U,V = sgd_factorise_masked(A, M, r, N, lr)
A_completed = U@V.t()
print(A_completed)

tensor([[0.2666, 0.2504, 0.0678],
        [0.5275, 0.4992, 0.1063],
        [1.6269, 1.5078, 0.5631]])
